In [1]:
import pandas as pd
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
premise_file = "../Resources/LicensePremise.csv"
premise_df = pd.read_csv(premise_file)
premise_df.head()

,License Serial Number,Premises Name,License Certificate Number,License Received Date,County ID Code
0,1311660,CANA ARRIBA GROCERY NO 2 INC,NaN,2018-06-29T00:00:00,0
1,1310023,JOHANA GROCERY & DELI CORP,NaN,2018-04-27T00:00:00,0
2,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1
3,1310024,TKO BEVERAGES LLC,NaN,2018-04-27T00:00:00,1
4,1311663,181 LEXINGTON AVENUE BBQ LLC,NaN,2018-06-29T00:00:00,2


In [3]:
county_file = "../Resources/CountyLicenseCount.csv"
county_df = pd.read_csv(county_file)
county_df.head()

,ID,County Name (Licensee),County ID Code,License Count
0,0,ALBANY,5,77
1,1,ALLEGANY,59,4
2,2,BRONX,0,104
3,3,BROOME,35,14
4,4,CATTARAUGUS,41,9


### Transform premise DataFrame

In [4]:
# Create a filtered dataframe from specific columns
premise_cols = ["License Serial Number", "Premises Name", "County ID Code"]
premise_transformed= premise_df[premise_cols].copy()

# Rename the column headers
premise_transformed = premise_transformed.rename(columns={"License Serial Number": "id",
                                                          "Premises Name": "premise_name",
                                                          "County ID Code": "county_id"})

# Clean the data by dropping duplicates and setting the index
premise_transformed.drop_duplicates("id", inplace=True)
premise_transformed.set_index("id", inplace=True)

premise_transformed.head()

,premise_name,county_id
id,,
1311660,CANA ARRIBA GROCERY NO 2 INC,0
1310023,JOHANA GROCERY & DELI CORP,0
1310024,TKO BEVERAGES LLC,1
1311663,181 LEXINGTON AVENUE BBQ LLC,2
1310029,AZIZ DELI & GRILL CORP,1


### Transform county DataFrame

In [5]:
county_cols = ["ID", "County Name (Licensee)", "County ID Code", "License Count"]
county_transformed = county_df[county_cols].copy()

# Rename the column headers
county_transformed = county_transformed.rename(columns={"ID": "id",
                                                         "County Name (Licensee)": "county_name",
                                                         "License Count": "license_count",
                                                         "County ID Code": "county_id"})

# Set index
county_transformed.set_index("id", inplace=True)

county_transformed.head()

,county_name,county_id,license_count
id,,,
0,ALBANY,5,77
1,ALLEGANY,59,4
2,BRONX,0,104
3,BROOME,35,14
4,CATTARAUGUS,41,9


### Create database connection

In [6]:
connection_string = "postgres:raw123@localhost:5432/customer_db"
engine = create_engine(f'postgresql://{connection_string}')

In [7]:
# Confirm tables
engine.table_names()

['customer_name', 'customer_location']

### Load DataFrames into database

In [21]:
premise_transformed.to_sql(name='premise', con=engine, if_exists='replace', index=True)

In [15]:
county_transformed.to_sql(name='county', con=engine, if_exists='append', index=True)

In [16]:
pd.read_sql_query('SELECT * FROM premise', con=engine).head()

,id,premise_name,county_id
0,1311660,CANA ARRIBA GROCERY NO 2 INC,0
1,1310023,JOHANA GROCERY & DELI CORP,0
2,1310024,TKO BEVERAGES LLC,1
3,1311663,181 LEXINGTON AVENUE BBQ LLC,2
4,1310029,AZIZ DELI & GRILL CORP,1


In [17]:
pd.read_sql_query('SELECT * FROM county', con=engine).head()

,id,county_name,license_count,county_id
0,0,ALBANY,77,5
1,1,ALLEGANY,4,59
2,2,BRONX,104,0
3,3,BROOME,14,35
4,4,CATTARAUGUS,9,41
